In [1]:
import keras
import tensorflow as tf

# Model Creation

## Fully Connected Network

In [21]:
INPUT_WIDTH = 224
INPUT_HEIGHT = 224
INPUT_CHANNELS = 3

### Backbone (VGG-16)

In [88]:
def build_vgg_backbone(input_shape=(INPUT_HEIGHT, INPUT_WIDTH, INPUT_CHANNELS)) -> keras.models.Sequential:
    keras.backend.clear_session()
    return keras.models.Sequential([
        # Conv1
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),

        # Conv2
        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),

        # Conv3
        keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="pool3"),

        # Conv4
        keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="pool4"),
        
        # Conv5
        keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    ], name="VGG 16")

In [89]:
build_vgg_backbone().summary()

Model: "VGG 16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

### FCN-32

In [80]:
NUM_CLASSES = 20
fcn32_backbone = build_vgg_backbone()
fcn32 = keras.models.Sequential([
    fcn32_backbone,
    # Conv7
    keras.layers.Conv2D(filters=4096, kernel_size=(7, 7), padding='same', activation='relu'),

    # Conv8
    keras.layers.Conv2D(filters=4096, kernel_size=(1, 1), padding='same', activation='relu'),

    # Conv9 fr_score
    keras.layers.Conv2D(filters=NUM_CLASSES, kernel_size=(1, 1), padding='same'),

    # DeconvConv10
    keras.layers.Conv2DTranspose(filters=NUM_CLASSES, kernel_size=(64, 64), strides=32),
    keras.layers.Activation("softmax") ## Might need to change depending on how data is structured
])

(224, 224, 3)


In [81]:
fcn32.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ VGG 16 (Sequential)             │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 7, 7, 4096)     │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 7, 7, 4096)     │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 7, 7, 20)       │        81,940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 256, 256, 20)   │     1,638,420 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256, 256, 20)   │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,980,904 (518.73 MB)

 Trainable params: 135,980,904 (518.73 MB)

 Non-trainable params: 0 (0.00 B)

### FCN-16

In [90]:
def build_fcn16(input_shape=(INPUT_HEIGHT, INPUT_WIDTH, INPUT_CHANNELS), num_classes=NUM_CLASSES):
    inputs = keras.Input(shape=input_shape)
    fcn16_backbone = build_vgg_backbone()
    pool4 = None
    x = inputs
    for layer in fcn16_backbone.layers:
        x = layer(x)
        if layer.name == "pool4":
            pool4 = x
    
    x = keras.layers.Conv2D(filters=4096, kernel_size=(7, 7), padding='same', activation='relu')(x)

    x = keras.layers.Conv2D(filters=4096, kernel_size=(1, 1), padding='same', activation='relu')(x)

    score_fr = keras.layers.Conv2D(num_classes, 1, padding='same')(x)

    score_fr_up = keras.layers.Conv2DTranspose(filters=num_classes, kernel_size=(4, 4), strides=2, padding='same')(score_fr)

    score_pool4 = keras.layers.Conv2D(filters=num_classes, kernel_size=(1, 1), padding='same')(pool4)

    fuse = keras.layers.Add()([score_fr_up, score_pool4])

    upsample = keras.layers.Conv2DTranspose(filters=num_classes, kernel_size=(32, 32), strides=16, padding='same')(fuse)

    outputs = keras.layers.Activation('softmax')(upsample)
    
    return keras.Model(inputs, outputs, name="FCN 16")

fcn16 = build_fcn16()


In [91]:
fcn16.summary()

Model: "FCN 16"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │     36,928 │ conv2d[1][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ conv2d_1[1][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │     73,856 │ max_pooling2d[1]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │    147,584 │ conv2d_2[1][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ conv2d_3[1][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 56, 56,    │    590,080 │ conv2d_4[1][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 56, 56,    │    590,080 │ conv2d_5[1][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool3               │ (None, 28, 28,    │          0 │ conv2d_6[1][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 28, 28,    │  1,180,160 │ pool3[1][0]       │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 28, 28,    │  2,359,808 │ conv2d_7[1][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 28, 28,    │  2,359,808 │ conv2d_8[1][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool4               │ (None, 14, 14,    │          0 │ conv2d_9[1][0]    │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 14, 14,    │  2,359,808 │ pool4[1][0]       │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 14, 14,    │  2,359,808 │ conv2d_10[1][0] 

 Total params: 134,768,784 (514.10 MB)

 Trainable params: 134,768,784 (514.10 MB)

 Non-trainable params: 0 (0.00 B)

### FCN 8

In [92]:
def build_fcn8(input_shape=(INPUT_HEIGHT, INPUT_WIDTH, INPUT_CHANNELS), num_classes=NUM_CLASSES):
    inputs = keras.Input(shape=input_shape)
    fcn16_backbone = build_vgg_backbone()
    pool3 = None
    pool4 = None
    x = inputs
    for layer in fcn16_backbone.layers:
        x = layer(x)
        if layer.name == "pool3":
            pool3 = x
        if layer.name == "pool4":
            pool4 = x
    
    x = keras.layers.Conv2D(filters=4096, kernel_size=(7, 7), padding='same', activation='relu')(x)

    x = keras.layers.Conv2D(filters=4096, kernel_size=(1, 1), padding='same', activation='relu')(x)

    score_fr = keras.layers.Conv2D(filters=num_classes, kernel_size=(1, 1), padding='same')(x)

    upscore2 = keras.layers.Conv2DTranspose(filters=num_classes, kernel_size=(4, 4), strides=2, padding='same')(score_fr)

    score_pool4 = keras.layers.Conv2D(filters=num_classes, kernel_size=(1, 1), padding='same')(pool4)

    fuse_pool4 = keras.layers.Add()([upscore2, score_pool4])

    upscore_pool4 = keras.layers.Conv2DTranspose(filters=num_classes, kernel_size=(4, 4), strides=2, padding='same')(fuse_pool4)

    score_pool3 = keras.layers.Conv2D(filters=num_classes, kernel_size=(1, 1), padding='same')(pool3)

    fuse_pool3 =  keras.layers.Add()([upscore_pool4, score_pool3])
    
    upscore8 = keras.layers.Conv2DTranspose(filters=num_classes, kernel_size=(16, 16), strides=8, padding='same')(fuse_pool3)

    outputs = keras.layers.Activation('softmax')(upscore8)
    
    return keras.Model(inputs, outputs, name="FCN 8")

fcn8 = build_fcn8()


In [86]:
fcn8.summary()

Model: "FCN 8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │     36,928 │ conv2d[1][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ conv2d_1[1][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │     73,856 │ max_pooling2d[1]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │    147,584 │ conv2d_2[1][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ conv2d_3[1][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 56, 56,    │    590,080 │ conv2d_4[1][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 56, 56,    │    590,080 │ conv2d_5[1][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 28, 28,    │          0 │ conv2d_6[1][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 28, 28,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 28, 28,    │  2,359,808 │ conv2d_7[1][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 28, 28,    │  2,359,808 │ conv2d_8[1][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 14, 14,    │          0 │ conv2d_9[1][0]    │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 14, 14,    │  2,359,808 │ max_pooling2d_3[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 14, 14,    │  2,359,808 │ conv2d_10[1][0] 

 Total params: 134,473,144 (512.97 MB)

 Trainable params: 134,473,144 (512.97 MB)

 Non-trainable params: 0 (0.00 B)

## Model Training Setup

### Optimiser, Loss Function, and Metrics

In [93]:
LEARNING_RATE = 0.01
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.9

In [95]:
optimiser = keras.optimizers.RMSprop(learning_rate=0.01, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
loss = keras.losses.CategoricalCrossentropy()
metrics = [keras.metrics.IoU(num_classes=NUM_CLASSES, target_class_ids=(1, 2, 3))]

# Will most likely need to change, also not worked with IoU so will need to look at

### Compiling Models

In [96]:
fcn32.compile(
    optimizer=optimiser,
    loss=loss,
    metrics=metrics
)
fcn16.compile(
    optimizer=optimiser,
    loss=loss,
    metrics=metrics
)
fcn8.compile(
    optimizer=optimiser,
    loss=loss,
    metrics=metrics
)

In [ ]:
# Dummy variables
train_ds = tf.data.Dataset()
validation_ds = tf.data.Dataset()
EPOCHS = 100
CALLBACKS = [keras.callbacks.EarlyStopping(monitor="val_loss")] 


### Training FCN 32

In [ ]:
history = fcn32.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=validation_ds,
    callbacks=CALLBACKS
)

### Training FCN 16

### Training FCN 8